In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install datasets
!pip install transformers
!pip install --upgrade transformers
!pip install einops
!pip install trl
!pip install huggingface_hub
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from unsloth import FastLanguageModel, PatchDPOTrainer
from trl import DPOConfig, DPOTrainer

# Clearing GPU memory cache
torch.cuda.empty_cache()

# Set model device type to 'cuda'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Using unsloth fast library
model, tokenizer =  FastLanguageModel.from_pretrained("unsloth/Llama-3.2-3B", dtype=None)
model = FastLanguageModel.get_peft_model(model)

tokenizer.pad_token = tokenizer.eos_token

# Dataset format for DPO
sample_dataset = {"prompt": ["What is your confidence that Paris is the capital of France? Respond with a percentage"], "chosen": [" 90%."], "rejected": [" 50%."]}

train_dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")
# train_dataset.set_format("torch", device="CPU")

# Patching the trainer
PatchDPOTrainer()

training_args = DPOConfig(output_dir="Llama3.2_3B-DPO", 
                          logging_steps=10, 
                          bf16=True,
                          per_device_train_batch_size=4,  
                          gradient_accumulation_steps=16)

trainer = DPOTrainer(model=model, 
                     args=training_args, 
                     processing_class=tokenizer, 
                     train_dataset=train_dataset)

print("Starting training...")

trainer.train()

# Save trained model
save_dir = "Llama3.2_3B_DPOtrained"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Trained model has been saved to {save_dir}")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# Set model device to 'cuda'
torch.set_default_device("cuda")

# Initialize model and corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained("./Llama3.2_3B_DPOtrained")
model = AutoModelForCausalLM.from_pretrained("./Llama3.2_3B_DPOtrained")


In [ ]:
import numpy as np

prompt = "There is most likely going to be fog around\n A. a marsh B. a tundra C. the plains D. a desert\n Respond and give your confidence in percentage terms. In the format, Answer-(a letter option) and Confidence-(a percent figure)"

inputs = tokenizer(prompt, return_tensors="pt")
inputs = inputs.to("cuda")

output = model.generate(**inputs, max_new_tokens=50, 
                         do_sample=False, 
                         temperature=1.0, 
                         output_logits=True,
                         return_dict_in_generate=True)

logits = output.logits

input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]

generated_tokens = output.sequences[:, input_length:]

# Apply softmax function to logits tensors
log_probs = torch.stack(logits, dim=1).softmax(-1)

# Extract probability that corresponds with generated tokens based on the softmax output
gen_probs = torch.gather(log_probs, 2, generated_tokens[:, :, None]).squeeze(-1)

token_probs = {}

for tok, score in zip(generated_tokens[0], gen_probs[0]):
    # Format token string | probabilityZzz
    # print(f" {tokenizer.decode(tok)} | {score.cpu().numpy()}")
    token_probs[tokenizer.decode(tok)] = score.cpu().numpy() * 100
    
output_answer = tokenizer.batch_decode(output.sequences, skip_special_tokens=True)[0]

print(output_answer)